In [1]:
# Import librairies

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression

import plotly.express as px

In [2]:
# URL of the CSV file of INSEE data
insee_url = 'https://medical-deserts-project.s3.eu-north-1.amazonaws.com/insee_clean.csv'

# Read the CSV file from the URL into a DataFrame
insee_df_original = pd.read_csv(insee_url, sep = ',', encoding='utf-8')

In [3]:
insee_df = insee_df_original.copy()

In [4]:
pd.set_option("display.max_columns", None)

In [5]:
# Remove useless columns
insee_df = insee_df.drop(["APL aux médecins généralistes de 65 ans et moins", "APL aux médecins généralistes de 62 ans et moins"], axis=1)

# APL column at the end of dataset
insee_df["SEG Croissance POP"] = insee_df.pop("SEG Croissance POP")
insee_df["Urbanité Ruralité"] = insee_df.pop("Urbanité Ruralité")
insee_df["Dynamique Démographique BV"] = insee_df.pop("Dynamique Démographique BV")
insee_df["Environnement Démographique"] = insee_df.pop("Environnement Démographique")
insee_df["Fidélité"] = insee_df.pop("Fidélité")
insee_df["Seg Cap Fiscale"] = insee_df.pop("Seg Cap Fiscale")
insee_df["Seg Dyn Entre"] = insee_df.pop("Seg Dyn Entre")
insee_df["DYN SetC"] = insee_df.pop("DYN SetC")
insee_df["Catégorie commune dans aire d'attraction des villes 2020"] = insee_df.pop("Catégorie commune dans aire d'attraction des villes 2020")
insee_df["Tranche détaillée d'aire d'attraction des villes 2020"] = insee_df.pop("Tranche détaillée d'aire d'attraction des villes 2020")
insee_df["Libellé degré de densité"] = insee_df.pop("Libellé degré de densité")
insee_df["APL aux médecins généralistes (sans borne d'âge)"] = insee_df.pop("APL aux médecins généralistes (sans borne d'âge)")

# Rename APL column
insee_df.rename(columns={"APL aux médecins généralistes (sans borne d'âge)": "APL"}, inplace=True)


In [6]:
print('Number of rows :', insee_df.shape[0])
print('Number of columns :', insee_df.shape[1])
print()

# Show first rows of the dataset
print('First rows of the dataset :')
display(insee_df.head())
print()

# Dataset statistics
print('Basics statistics :')
summary_stats_all = insee_df.describe(include='all')
display(summary_stats_all)
print()

# Missing values percentage
missing_percentages = (insee_df.isna().mean() * 100).round(2)
print('Percentage of missing values: ')
print(missing_percentages)

Number of rows : 38590
Number of columns : 88

First rows of the dataset :


,Dynamique Entrepreneuriale,Dynamique Entrepreneuriale Service et Commerce,Synergie Médicale COMMUNE,Nb Omnipraticiens BV,Nb Infirmiers Libéraux BV,Nb dentistes Libéraux BV,Nb pharmaciens Libéraux BV,Densité Médicale BV,Score équipement de santé BV,Indice Démographique,Nb Ménages,Nb Résidences Principales,Nb propriétaire,Nb Logement,Nb Résidences Secondaires,Nb Log Vacants,Nb Occupants Résidence Principale,Nb Entreprises Secteur Services,Nb Entreprises Secteur Commerce,Nb Entreprises Secteur Construction,Nb Entreprises Secteur Industrie,Nb Création Enteprises,Nb Création Industrielles,Nb Création Construction,Nb Création Commerces,Nb Création Services,Moyenne Revenus Fiscaux Départementaux,Moyenne Revenus Fiscaux Régionaux,Dep Moyenne Salaires Horaires,Dep Moyenne Salaires Cadre Horaires,Dep Moyenne Salaires Prof Intermédiaire Horaires,Dep Moyenne Salaires Employé Horaires,Dep Moyenne Salaires Ouvrié Horaires,Reg Moyenne Salaires Horaires,Reg Moyenne Salaires Cadre Horaires,Reg Moyenne Salaires Prof Intermédiaire Horaires,Reg Moyenne Salaires Employé Horaires,Reg Moyenne Salaires Ouvrié Horaires,Valeur ajoutée régionale,Nb Logement Secondaire et Occasionnel,Nb Hotel,Capacité Hotel,Nb Camping,Capacité Camping,Taux Propriété,Dynamique Démographique INSEE,Capacité Fisc,Moyenne Revnus fiscaux,"Nb Education, santé, action sociale",Nb Services personnels et domestiques,"Nb Santé, action sociale",Nb Industries des biens intermédiaires,Nb de Commerce,Nb de Services aux particuliers,"Nb institution de Education, santé, action sociale, administration",PIB Régionnal,Score Croissance Entrepreneuriale,latitude,longitude,Population en 2014 (princ),Pop 0-14 ans en 2014 (princ),Pop 15-29 ans en 2014 (princ),Pop 30-44 ans en 2014 (princ),Pop 45-59 ans en 2014 (princ),Pop 60-74 ans en 2014 (princ),Pop 75-89 ans en 2014 (princ),Pop 15 ans ou plus en 2014 (compl),Pop 15 ans ou plus Agriculteurs exploitants en 2014 (compl),"Pop 15 ans ou plus Artisans, Comm., Chefs entr. en 2014 (compl)","Pop 15 ans ou plus Cadres, Prof. intel. sup. en 2014 (compl)",Pop 15 ans ou plus Prof. intermédiaires en 2014 (compl),Pop 15 ans ou plus Employés en 2014 (compl),Pop 15 ans ou plus Ouvriers en 2014 (compl),Pop 15 ans ou plus Retraités en 2014 (compl),Pop 15 ans ou plus Autres en 2014 (compl),taux chômage(15-64 ans),SEG Croissance POP,Urbanité Ruralité,Dynamique Démographique BV,Environnement Démographique,Fidélité,Seg Cap Fiscale,Seg Dyn Entre,DYN SetC,Catégorie commune dans aire d'attraction des villes 2020,Tranche détaillée d'aire d'attraction des villes 2020,Libellé degré de densité,APL
0,57.0,23.0,114,9,14,7,7,0.092860,4,44.19769,247,248,196,289,32,9,728,7.0,11.0,2.0,2.0,4.0,0.0,2.0,1.0,1.0,12509,10458,11.410345,21.963793,12.558621,8.743103,9.268966,11.873022,21.78783,12.704057,8.783164,9.30142,86957.45836,32.0,0.0,0.0,0.0,0.0,67,-1,117,11483.5,3.0,1.0,0.0,9364,9350,3372,15105,173681,0.01585,46.153721,4.925850,767.0,161.0,102.0,132.0,189.0,125.0,53.0,605.0,15.0,20.0,75.0,95.0,100.0,125.0,145.0,30.0,8.776596,en croissance démographique,Com rurale < 2 000 m habts,1.Accroissement par excédent naturel et migrat...,Bassin Industriel en croissance démographique,Pop Sédentaire,Fiscalité moyenne,Faible dynamique,Faible Dynamique Serv et Com,Commune de la couronne,Aire de moins de 10 000 habitants,Rural à habitat dispersé,2.293
1,45.0,4.0,143,31,36,18,18,0.099229,4,10.18071,67,67,61,142,71,4,168,4.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,12509,10458,11.410345,21.963793,12.558621,8.743103,9.268966,11.873022,21.78783,12.704057,8.783164,9.30142,86957.45836,71.0,0.0,0.0,0.0,0.0,42,0,110,11483.5,0.0,0.0,0.0,9364,9350,3372,15105,173681,0.00173,46.009606,5.428088,239.0,54.0,28.0,69.0,36.0,33.0,17.0,192.0,0.0,20.0,15.0,25.0,39.0,10.0,64.0,20.0,8.130081,en croissance démographique,Com rurale < 2 000 m habts,1.Accroissement par excédent naturel et migrat...,Bassin Résidentiel en croissance démographique,Pop Sédentaire,Fiscalité moyenne,Faible dynamique,Faible Dynamique Serv et Com,Com


Basics statistics :


,Dynamique Entrepreneuriale,Dynamique Entrepreneuriale Service et Commerce,Synergie Médicale COMMUNE,Nb Omnipraticiens BV,Nb Infirmiers Libéraux BV,Nb dentistes Libéraux BV,Nb pharmaciens Libéraux BV,Densité Médicale BV,Score équipement de santé BV,Indice Démographique,Nb Ménages,Nb Résidences Principales,Nb propriétaire,Nb Logement,Nb Résidences Secondaires,Nb Log Vacants,Nb Occupants Résidence Principale,Nb Entreprises Secteur Services,Nb Entreprises Secteur Commerce,Nb Entreprises Secteur Construction,Nb Entreprises Secteur Industrie,Nb Création Enteprises,Nb Création Industrielles,Nb Création Construction,Nb Création Commerces,Nb Création Services,Moyenne Revenus Fiscaux Départementaux,Moyenne Revenus Fiscaux Régionaux,Dep Moyenne Salaires Horaires,Dep Moyenne Salaires Cadre Horaires,Dep Moyenne Salaires Prof Intermédiaire Horaires,Dep Moyenne Salaires Employé Horaires,Dep Moyenne Salaires Ouvrié Horaires,Reg Moyenne Salaires Horaires,Reg Moyenne Salaires Cadre Horaires,Reg Moyenne Salaires Prof Intermédiaire Horaires,Reg Moyenne Salaires Employé Horaires,Reg Moyenne Salaires Ouvrié Horaires,Valeur ajoutée régionale,Nb Logement Secondaire et Occasionnel,Nb Hotel,Capacité Hotel,Nb Camping,Capacité Camping,Taux Propriété,Dynamique Démographique INSEE,Capacité Fisc,Moyenne Revnus fiscaux,"Nb Education, santé, action sociale",Nb Services personnels et domestiques,"Nb Santé, action sociale",Nb Industries des biens intermédiaires,Nb de Commerce,Nb de Services aux particuliers,"Nb institution de Education, santé, action sociale, administration",PIB Régionnal,Score Croissance Entrepreneuriale,latitude,longitude,Population en 2014 (princ),Pop 0-14 ans en 2014 (princ),Pop 15-29 ans en 2014 (princ),Pop 30-44 ans en 2014 (princ),Pop 45-59 ans en 2014 (princ),Pop 60-74 ans en 2014 (princ),Pop 75-89 ans en 2014 (princ),Pop 15 ans ou plus en 2014 (compl),Pop 15 ans ou plus Agriculteurs exploitants en 2014 (compl),"Pop 15 ans ou plus Artisans, Comm., Chefs entr. en 2014 (compl)","Pop 15 ans ou plus Cadres, Prof. intel. sup. en 2014 (compl)",Pop 15 ans ou plus Prof. intermédiaires en 2014 (compl),Pop 15 ans ou plus Employés en 2014 (compl),Pop 15 ans ou plus Ouvriers en 2014 (compl),Pop 15 ans ou plus Retraités en 2014 (compl),Pop 15 ans ou plus Autres en 2014 (compl),taux chômage(15-64 ans),SEG Croissance POP,Urbanité Ruralité,Dynamique Démographique BV,Environnement Démographique,Fidélité,Seg Cap Fiscale,Seg Dyn Entre,DYN SetC,Catégorie commune dans aire d'attraction des villes 2020,Tranche détaillée d'aire d'attraction des villes 2020,Libellé degré de densité,APL
count,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590,38590,38590,38590,38590,38590,38590,38590,38590,38590,38590,38590.000000
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,6,7,7,2,3,3,3,


Percentage of missing values: 
Dynamique Entrepreneuriale                                  0.0
Dynamique Entrepreneuriale Service et Commerce              0.0
Synergie Médicale COMMUNE                                   0.0
Nb Omnipraticiens BV                                        0.0
Nb Infirmiers Libéraux BV                                   0.0
                                                           ... 
DYN SetC                                                    0.0
Catégorie commune dans aire d'attraction des villes 2020    0.0
Tranche détaillée d'aire d'attraction des villes 2020       0.0
Libellé degré de densité                                    0.0
APL                                                         0.0
Length: 88, dtype: float64


In [7]:
# Separate target variable Y from features X
print("Separating labels from features...")
target_variable = "APL"

X = insee_df.drop(target_variable, axis = 1)
Y = insee_df.loc[:,target_variable]

print("...Done.")
print()

print('Y : ')
print(Y.head())
print()
print('X :')
X.head()

Separating labels from features...
...Done.

Y : 
0    2.293
1    2.600
2    4.079
3    4.378
4    1.069
Name: APL, dtype: float64

X :


,Dynamique Entrepreneuriale,Dynamique Entrepreneuriale Service et Commerce,Synergie Médicale COMMUNE,Nb Omnipraticiens BV,Nb Infirmiers Libéraux BV,Nb dentistes Libéraux BV,Nb pharmaciens Libéraux BV,Densité Médicale BV,Score équipement de santé BV,Indice Démographique,Nb Ménages,Nb Résidences Principales,Nb propriétaire,Nb Logement,Nb Résidences Secondaires,Nb Log Vacants,Nb Occupants Résidence Principale,Nb Entreprises Secteur Services,Nb Entreprises Secteur Commerce,Nb Entreprises Secteur Construction,Nb Entreprises Secteur Industrie,Nb Création Enteprises,Nb Création Industrielles,Nb Création Construction,Nb Création Commerces,Nb Création Services,Moyenne Revenus Fiscaux Départementaux,Moyenne Revenus Fiscaux Régionaux,Dep Moyenne Salaires Horaires,Dep Moyenne Salaires Cadre Horaires,Dep Moyenne Salaires Prof Intermédiaire Horaires,Dep Moyenne Salaires Employé Horaires,Dep Moyenne Salaires Ouvrié Horaires,Reg Moyenne Salaires Horaires,Reg Moyenne Salaires Cadre Horaires,Reg Moyenne Salaires Prof Intermédiaire Horaires,Reg Moyenne Salaires Employé Horaires,Reg Moyenne Salaires Ouvrié Horaires,Valeur ajoutée régionale,Nb Logement Secondaire et Occasionnel,Nb Hotel,Capacité Hotel,Nb Camping,Capacité Camping,Taux Propriété,Dynamique Démographique INSEE,Capacité Fisc,Moyenne Revnus fiscaux,"Nb Education, santé, action sociale",Nb Services personnels et domestiques,"Nb Santé, action sociale",Nb Industries des biens intermédiaires,Nb de Commerce,Nb de Services aux particuliers,"Nb institution de Education, santé, action sociale, administration",PIB Régionnal,Score Croissance Entrepreneuriale,latitude,longitude,Population en 2014 (princ),Pop 0-14 ans en 2014 (princ),Pop 15-29 ans en 2014 (princ),Pop 30-44 ans en 2014 (princ),Pop 45-59 ans en 2014 (princ),Pop 60-74 ans en 2014 (princ),Pop 75-89 ans en 2014 (princ),Pop 15 ans ou plus en 2014 (compl),Pop 15 ans ou plus Agriculteurs exploitants en 2014 (compl),"Pop 15 ans ou plus Artisans, Comm., Chefs entr. en 2014 (compl)","Pop 15 ans ou plus Cadres, Prof. intel. sup. en 2014 (compl)",Pop 15 ans ou plus Prof. intermédiaires en 2014 (compl),Pop 15 ans ou plus Employés en 2014 (compl),Pop 15 ans ou plus Ouvriers en 2014 (compl),Pop 15 ans ou plus Retraités en 2014 (compl),Pop 15 ans ou plus Autres en 2014 (compl),taux chômage(15-64 ans),SEG Croissance POP,Urbanité Ruralité,Dynamique Démographique BV,Environnement Démographique,Fidélité,Seg Cap Fiscale,Seg Dyn Entre,DYN SetC,Catégorie commune dans aire d'attraction des villes 2020,Tranche détaillée d'aire d'attraction des villes 2020,Libellé degré de densité
0,57.0,23.0,114,9,14,7,7,0.092860,4,44.19769,247,248,196,289,32,9,728,7.0,11.0,2.0,2.0,4.0,0.0,2.0,1.0,1.0,12509,10458,11.410345,21.963793,12.558621,8.743103,9.268966,11.873022,21.78783,12.704057,8.783164,9.30142,86957.45836,32.0,0.0,0.0,0.0,0.0,67,-1,117,11483.5,3.0,1.0,0.0,9364,9350,3372,15105,173681,0.01585,46.153721,4.925850,767.0,161.0,102.0,132.0,189.0,125.0,53.0,605.0,15.0,20.0,75.0,95.0,100.0,125.0,145.0,30.0,8.776596,en croissance démographique,Com rurale < 2 000 m habts,1.Accroissement par excédent naturel et migrat...,Bassin Industriel en croissance démographique,Pop Sédentaire,Fiscalité moyenne,Faible dynamique,Faible Dynamique Serv et Com,Commune de la couronne,Aire de moins de 10 000 habitants,Rural à habitat dispersé
1,45.0,4.0,143,31,36,18,18,0.099229,4,10.18071,67,67,61,142,71,4,168,4.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,12509,10458,11.410345,21.963793,12.558621,8.743103,9.268966,11.873022,21.78783,12.704057,8.783164,9.30142,86957.45836,71.0,0.0,0.0,0.0,0.0,42,0,110,11483.5,0.0,0.0,0.0,9364,9350,3372,15105,173681,0.00173,46.009606,5.428088,239.0,54.0,28.0,69.0,36.0,33.0,17.0,192.0,0.0,20.0,15.0,25.0,39.0,10.0,64.0,20.0,8.130081,en croissance démographique,Com rurale < 2 000 m habts,1.Accroissement par excédent naturel et migrat...,Bassin Résidentiel en croissance démographique,Pop Sédentaire,Fiscalité moyenne,Faible dynamique,Faible Dynamique Serv et Com,Commune hors 

In [9]:
#Train-test splitting

print("Dividing into train and test sets...")
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
print("...Done.")
print()

Dividing into train and test sets...


...Done.



In [10]:
# Automatically detect names of numeric/categorical columns
numeric_features = []
categorical_features = []
for i, t in X.dtypes.items():
    if ('float' in str(t)) or ('int' in str(t)):
        numeric_features.append(i)
    else:
        categorical_features.append(i)
print('Found numeric features ', numeric_features)
print('Found categorical features ', categorical_features)

Found numeric features  ['Dynamique Entrepreneuriale', 'Dynamique Entrepreneuriale Service et Commerce', 'Synergie Médicale COMMUNE', 'Nb Omnipraticiens BV', 'Nb Infirmiers Libéraux BV', 'Nb dentistes Libéraux BV', 'Nb pharmaciens Libéraux BV', 'Densité Médicale BV', 'Score équipement de santé BV', 'Indice Démographique', 'Nb Ménages', 'Nb Résidences Principales', 'Nb propriétaire', 'Nb Logement', 'Nb Résidences Secondaires', 'Nb Log Vacants', 'Nb Occupants Résidence Principale', 'Nb Entreprises Secteur Services', 'Nb Entreprises Secteur Commerce', 'Nb Entreprises Secteur Construction', 'Nb Entreprises Secteur Industrie', 'Nb Création Enteprises', 'Nb Création Industrielles', 'Nb Création Construction', 'Nb Création Commerces', 'Nb Création Services', 'Moyenne Revenus Fiscaux Départementaux', 'Moyenne Revenus Fiscaux Régionaux', 'Dep Moyenne Salaires Horaires', 'Dep Moyenne Salaires Cadre Horaires', 'Dep Moyenne Salaires Prof Intermédiaire Horaires', 'Dep Moyenne Salaires Employé Horai

In [11]:
#Multiple Transformations preprocessing

# Create pipeline for numeric features
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')), # missing values will be replaced by columns' mean
    ('scaler', StandardScaler())
])

# Create pipeline for categorical features
categorical_transformer = Pipeline(
    steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), # missing values will be replaced by most frequent value
    ('encoder', OneHotEncoder(drop='first', handle_unknown='ignore')) # first column will be dropped to avoid creating correlations between features
    ])

# Use ColumnTransformer to make a preprocessor object that describes all the treatments to be done
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

In [12]:
# Preprocessings on train set
print("Performing preprocessings on train set...")
print(X_train.head())
X_train = preprocessor.fit_transform(X_train)
print('...Done.')
print(X_train[0:5,:])
print()

# Preprocessings on test set
print("Performing preprocessings on test set...")
print(X_test.head())
X_test = preprocessor.transform(X_test) # Don't fit again !!
print('...Done.')
print(X_test[0:5,:])
print()

Performing preprocessings on train set...
       Dynamique Entrepreneuriale  \
19078                        41.0   
7714                         40.0   
3753                         11.0   
10173                        63.0   
26517                        70.0   

       Dynamique Entrepreneuriale Service et Commerce  \
19078                                            16.0   
7714                                             14.0   
3753                                              1.0   
10173                                            10.0   
26517                                            55.0   

       Synergie Médicale COMMUNE  Nb Omnipraticiens BV  \
19078                         77                    12   
7714                          46                    10   
3753                          33                     8   
10173                         97                     4   
26517                         99                    12   

       Nb Infirmiers Libéraux BV  Nb dentis

In [13]:
column_names = preprocessor.get_feature_names_out()

In [14]:
# Train model
model = LinearRegression()
print("Training model...")
model.fit(X_train, Y_train)  # Training is always done on train set !!
print("...Done.")

Training model...


...Done.


In [15]:
# Print R^2 scores
print("R2 score on training set : ", model.score(X_train, Y_train))
print("R2 score on test set : ", model.score(X_test, Y_test))

R2 score on training set :  0.2409690103939559
R2 score on test set :  0.24180466141172086


## Forward selection

In [16]:
from sklearn.feature_selection import  SequentialFeatureSelector
feature_selector =  SequentialFeatureSelector(model, n_features_to_select = 10)
feature_selector.fit(X_train, Y_train)
#features_list = X.columns
#best_features = features_list[feature_selector.support_]
print("According to the forward selection algorithm, the following features should be kept: ")
print(feature_selector.support_.tolist())


In [ ]:
# Get the indices of the selected features
selected_feature_indices = feature_selector.get_support(indices=True)

# Use these indices to filter the corresponding column names
selected_column_names = [column_names[i] for i in selected_feature_indices]

# Print the names of the selected features
print("The names of the selected features are:", selected_column_names)

The names of the selected features are: ['num__Score équipement de santé BV', 'num__Moyenne Revenus Fiscaux Départementaux', 'num__Reg Moyenne Salaires Prof Intermédiaire Horaires', 'num__Nb Hotel', 'num__Taux Propriété', 'cat__Urbanité Ruralité_Com < 200 m habts', 'cat__Urbanité Ruralité_Com > 200 m habts', 'cat__Urbanité Ruralité_Com rurale < 2 000 m habts', 'cat__Libellé degré de densité_Rural à habitat dispersé', 'cat__Libellé degré de densité_Rural à habitat très dispersé']


In [ ]:
# Initialize the filtered list
temp_final_features = []
final_features = []

# Iterate through the example list
for element in selected_column_names:
    if element.startswith("num__"):
        # If the element starts with "num__", keep it as is
        temp_final_features.append(element)
    elif element.startswith("cat__"):
        # If the element starts with "cat__", find the index of the last underscore
        last_underscore_index = element.rfind('_')
        # Keep only the part of the element up to the last underscore
        filtered_element = element[:last_underscore_index]
        # Add the filtered element to the filtered list
        temp_final_features.append(filtered_element)

for element in temp_final_features:
    # Checkif the element is already in the list
    if element not in final_features:
        # If the element is not in the list, add it
        final_features.append(element)

final_features = [name[5:] for name in final_features]

# Print the filtered list
print("Filtered List:", final_features)


Filtered List: ['Score équipement de santé BV', 'Moyenne Revenus Fiscaux Départementaux', 'Reg Moyenne Salaires Prof Intermédiaire Horaires', 'Nb Hotel', 'Taux Propriété', 'Urbanité Ruralité', 'Libellé degré de densité']


In [ ]:
insee_best = insee_df[final_features + ["APL"]]


In [ ]:
print('Number of rows :', insee_best.shape[0])
print('Number of columns :', insee_best.shape[1])
print()

# Show first rows of the dataset
print('First rows of the dataset :')
display(insee_best.head())
print()

# Dataset statistics
print('Basics statistics :')
summary_stats_all = insee_best.describe(include='all')
display(summary_stats_all)
print()

# Missing values percentage
missing_percentages = (insee_best.isna().mean() * 100).round(2)
print('Percentage of missing values: ')
print(missing_percentages)

Number of rows : 38590
Number of columns : 8

First rows of the dataset :


,Score équipement de santé BV,Moyenne Revenus Fiscaux Départementaux,Reg Moyenne Salaires Prof Intermédiaire Horaires,Nb Hotel,Taux Propriété,Urbanité Ruralité,Libellé degré de densité,APL
0,4,12509,12.704057,0.0,67,Com rurale < 2 000 m habts,Rural à habitat dispersé,2.293
1,4,12509,12.704057,0.0,42,Com rurale < 2 000 m habts,Rural à habitat dispersé,2.600
2,4,12509,12.704057,2.0,37,Com < 50 m habts,Centres urbains intermédiaires,4.079
3,1,12509,12.704057,2.0,68,Com rurale < 2 000 m habts,Bourgs ruraux,4.378
4,3,12509,12.704057,0.0,49,Com rurale < 2 000 m habts,Rural à habitat dispersé,1.069



Basics statistics :


,Score équipement de santé BV,Moyenne Revenus Fiscaux Départementaux,Reg Moyenne Salaires Prof Intermédiaire Horaires,Nb Hotel,Taux Propriété,Urbanité Ruralité,Libellé degré de densité,APL
count,38590.000000,38590.000000,38590.000000,38590.000000,38590.000000,38590,38590,38590.000000
unique,NaN,NaN,NaN,NaN,NaN,6,7,NaN
top,NaN,NaN,NaN,NaN,NaN,Com rurale < 2 000 m habts,Rural à habitat dispersé,NaN
freq,NaN,NaN,NaN,NaN,NaN,30651,19922,NaN
mean,2.527961,10349.030319,12.519946,0.672687,57.411791,NaN,NaN,3.292126
std,1.320803,1564.368493,0.298912,4.141992,15.613730,NaN,NaN,1.279848
min,0.000000,6282.000000,12.175380,0.000000,0.000000,NaN,NaN,0.000000
25%,1.000000,9523.000000,12.339819,0.000000,48.000000,NaN,NaN,2.507000
50%,3.000000,10159.000000,12.435217,0.000000,59.000000,NaN,NaN,3.229000
75%,4.000000,11072.000000,12.673103,0.000000,69.000000,NaN,NaN,4.009000



Percentage of missing values: 
Score équipement de santé BV                        0.0
Moyenne Revenus Fiscaux Départementaux              0.0
Reg Moyenne Salaires Prof Intermédiaire Horaires    0.0
Nb Hotel                                            0.0
Taux Propriété                                      0.0
Urbanité Ruralité                                   0.0
Libellé degré de densité                            0.0
APL                                                 0.0
dtype: float64


In [ ]:
# Separate target variable Y from features X
print("Separating labels from features...")
target_variable = "APL"

X = insee_best.drop(target_variable, axis = 1)
Y = insee_best.loc[:,target_variable]

print("...Done.")
print()

print('Y : ')
print(Y.head())
print()
print('X :')
X.head()

Separating labels from features...
...Done.

Y : 
0    2.293
1    2.600
2    4.079
3    4.378
4    1.069
Name: APL, dtype: float64

X :


,Score équipement de santé BV,Moyenne Revenus Fiscaux Départementaux,Reg Moyenne Salaires Prof Intermédiaire Horaires,Nb Hotel,Taux Propriété,Urbanité Ruralité,Libellé degré de densité
0,4,12509,12.704057,0.0,67,Com rurale < 2 000 m habts,Rural à habitat dispersé
1,4,12509,12.704057,0.0,42,Com rurale < 2 000 m habts,Rural à habitat dispersé
2,4,12509,12.704057,2.0,37,Com < 50 m habts,Centres urbains intermédiaires
3,1,12509,12.704057,2.0,68,Com rurale < 2 000 m habts,Bourgs ruraux
4,3,12509,12.704057,0.0,49,Com rurale < 2 000 m habts,Rural à habitat dispersé


In [ ]:
#Train-test splitting

print("Dividing into train and test sets...")
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
print("...Done.")
print()

Dividing into train and test sets...
...Done.



In [ ]:
# Automatically detect names of numeric/categorical columns
numeric_features = []
categorical_features = []
for i, t in X.dtypes.items():
    if ('float' in str(t)) or ('int' in str(t)):
        numeric_features.append(i)
    else:
        categorical_features.append(i)
print('Found numeric features ', numeric_features)
print('Found categorical features ', categorical_features)

Found numeric features  ['Score équipement de santé BV', 'Moyenne Revenus Fiscaux Départementaux', 'Reg Moyenne Salaires Prof Intermédiaire Horaires', 'Nb Hotel', 'Taux Propriété']
Found categorical features  ['Urbanité Ruralité', 'Libellé degré de densité']


In [ ]:
#Multiple Transformations preprocessing

# Create pipeline for numeric features
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')), # missing values will be replaced by columns' mean
    ('scaler', StandardScaler())
])

# Create pipeline for categorical features
categorical_transformer = Pipeline(
    steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), # missing values will be replaced by most frequent value
    ('encoder', OneHotEncoder(drop='first', handle_unknown='ignore')) # first column will be dropped to avoid creating correlations between features
    ])

# Use ColumnTransformer to make a preprocessor object that describes all the treatments to be done
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

In [ ]:
# Preprocessings on train set
print("Performing preprocessings on train set...")
print(X_train.head())
X_train = preprocessor.fit_transform(X_train)
print('...Done.')
print(X_train[0:5,:])
print()

# Preprocessings on test set
print("Performing preprocessings on test set...")
print(X_test.head())
X_test = preprocessor.transform(X_test) # Don't fit again !!
print('...Done.')
print(X_test[0:5,:])
print()

Performing preprocessings on train set...
       Score équipement de santé BV  Moyenne Revenus Fiscaux Départementaux  \
19078                             1                                    9860   
7714                              3                                   14584   
3753                              2                                    9364   
10173                             1                                    6688   
26517                             1                                    8966   

       Reg Moyenne Salaires Prof Intermédiaire Horaires  Nb Hotel  \
19078                                         12.435217       0.0   
7714                                          12.430189       0.0   
3753                                          12.489024       0.0   
10173                                         12.704057       0.0   
26517                                         12.444562       1.0   

       Taux Propriété           Urbanité Ruralité  \
19078          

In [ ]:
# Train model
model = LinearRegression()
print("Training model...")
model.fit(X_train, Y_train)  # Training is always done on train set !!
print("...Done.")

Training model...
...Done.


In [ ]:
# Print R^2 scores
print("R2 score on training set : ", model.score(X_train, Y_train))
print("R2 score on test set : ", model.score(X_test, Y_test))

R2 score on training set :  0.1593025792632331
R2 score on test set :  0.14773348715170243


In [ ]:
column_names = []
for name, pipeline, features_list in preprocessor.transformers_: # loop over pipelines
    if name == 'num': # if pipeline is for numeric variables
        features = features_list # just get the names of columns to which it has been applied
    else: # if pipeline is for categorical variables
        features = pipeline.named_steps['encoder'].get_feature_names_out() # get output columns names from OneHotEncoder
    column_names.extend(features) # concatenate features names
        
print("Names of columns corresponding to each coefficient: ", column_names)

# Create a pandas DataFrame
coefs = pd.DataFrame(index = column_names, data = model.coef_.transpose(), columns=["coefficients"])
coefs

# Compute abs() and sort values
feature_importance = abs(coefs).sort_values(by = 'coefficients')
feature_importance

# Plot coefficients
fig = px.bar(feature_importance, orientation = 'h')
fig.update_layout(showlegend = False, 
                  margin = {'l': 120} # to avoid cropping of column names
                 )

fig.update_layout(height=2400, width=1600)

fig.show()

Names of columns corresponding to each coefficient:  ['Score équipement de santé BV', 'Moyenne Revenus Fiscaux Départementaux', 'Reg Moyenne Salaires Prof Intermédiaire Horaires', 'Nb Hotel', 'Taux Propriété', 'x0_Com < 200 m habts', 'x0_Com < 50 m habts', 'x0_Com > 200 m habts', 'x0_Com rurale < 2 000 m habts', 'x0_Com rurale > 2 000 habts', 'x1_Ceintures urbaines', 'x1_Centres urbains intermédiaires', 'x1_Grands centres urbains', 'x1_Petites villes', 'x1_Rural à habitat dispersé', 'x1_Rural à habitat très dispersé']
